<a href="https://colab.research.google.com/github/Igprad01/Sentimen-Analisis/blob/main/Analisis_Sentimen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ANALISIS SENTIMEN CASE STUDY OPINI PUBLIK TWITTER

INPUT DATA SET

In [114]:
import pandas as pd

# load data
files = '/content/sample_data/Gabungan sample crawling JOT jkt48.csv'
df = pd.read_csv(files)
df.head()
# display(df)

,conversation_id_str,created_at,favorite_count,full_text,id_str,image_url,in_reply_to_screen_name,lang,location,quote_count,reply_count,retweet_count,tweet_url,user_id_str,username
0,1.870000e+18,Mon Dec 30 18:55:01 +0000 2024,0,Om JOT Liat deh Si up girls ngajak @RFritzy_JK...,1.870000e+18,NaN,NaN,in,NaN,0,0,0,https://x.com/XJulianto/status/187380500346035...,1.820000e+18,XJulianto
1,1.870000e+18,Mon Dec 30 17:47:27 +0000 2024,0,@RFritzy_JKT48 Kalo oshinya JOT masih ga apa a...,1.870000e+18,https://pbs.twimg.com/media/GgEIPq1bYAExxgW.jpg,RFritzy_JKT48,in,"DKI Jakarta, Indonesia",0,0,0,https://x.com/Anggithaadr/status/1873788000431...,8.200000e+17,Anggithaadr
2,1.870000e+18,Mon Dec 30 15:52:30 +0000 2024,0,harapan untuk jkt48 yaitu membuat para jot sad...,1.870000e+18,NaN,NaN,in,NaN,0,0,0,https://x.com/blq_abilzz/status/18737590745775...,1.790000e+18,blq_abilzz
3,1.870000e+18,Mon Dec 30 15:25:31 +0000 2024,1,@_rybh @Maffi_f07 @Auwia_JKT48 Itu urusan inte...,1.870000e+18,NaN,_rybh,in,Salakanagara,0,1,0,https://x.com/gagitugie___/status/187375228321...,2.429397e+09,gagitugie___
4,1.870000e+18,Mon Dec 30 15:07:03 +0000 2024,0,@gagitugie___ @Maffi_f07 @Auwia_JKT48 Menunjuk...,1.870000e+18,NaN,gagitugie___,in,NaN,0,1,0,https://x.com/_rybh/status/1873747635750011333,3.181792e+09,_rybh


DETEKSI DUPLIKASI DATA

In [94]:
#duplicate data
# df_duplicates = df[df.duplicated('full_text')]
# display(df_duplicates)

,conversation_id_str,created_at,favorite_count,full_text,id_str,image_url,in_reply_to_screen_name,lang,location,quote_count,reply_count,retweet_count,tweet_url,user_id_str,username
36,1.870000e+18,Sun Dec 29 04:24:09 +0000 2024,0,@Nachia_JKT48 dapet siapa tuhh gacha pc semale...,1.870000e+18,NaN,Nachia_JKT48,in,jekate bukan 48,0,0,0,https://x.com/minjiabs/status/1873223456164401232,1.570000e+18,minjiabs
93,1.870000e+18,Fri Dec 27 06:33:01 +0000 2024,0,@wasitidol_2nd (1). tp aulia ragu krn mungkin ...,1.870000e+18,NaN,Fauziir12,in,NaN,0,0,0,https://x.com/Fauziir12/status/187253111285943...,1.860000e+18,Fauziir12
94,1.870000e+18,Fri Dec 27 06:29:08 +0000 2024,0,@wasitidol_2nd Ini pendapat gw pribadi yaa dan...,1.870000e+18,NaN,NagekiKun_,in,NaN,0,5,0,https://x.com/Fauziir12/status/187253013327137...,1.860000e+18,Fauziir12
249,1.870000e+18,Thu Dec 19 01:10:02 +0000 2024,3,@peyeeeek jadi begini sahabatku.. pd bln septe...,1.870000e+18,NaN,peyeeeek,in,NaN,0,1,0,https://x.com/cruzozzan/status/186955072856772...,1.280000e+18,cruzozzan
250,1.870000e+18,Thu Dec 19 01:04:15 +0000 2024,4,@peyeeeek jadi begini sahabatku.. pd bln septe...,1.870000e+18,NaN,peyeeeek,in,Ig. Bayuec_,0,0,0,https://x.com/BayuEkaCndra/status/186954927272...,1.280000e+18,BayuEkaCndra
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6211,1.750000e+18,Fri Jan 19 11:29:04 +0000 2024,1,Lahkk makasih om jot See u @Greesel_JKT48 http...,1.750000e+18,https://pbs.twimg.com/media/GEM7jzMasAApT7x.jpg,NaN,in,Indonesia,0,0,0,https://x.com/R_Hafidzmuh/status/1748306574794...,1.210000e+18,R_Hafidzmuh
6212,1.750000e+18,Fri Jan 19 11:14:00 +0000 2024,1,@JambuBiji08 @Gracie_JKT48 Harusnya faham JOT om,1.750000e+18,NaN,JambuBiji08,in,NaN,0,1,0,https://x.com/zibarrr_/status/1748302786700439566,1.600000e+18,zibarrr_
6213,1.750000e+18,Fri Jan 19 08:35:25 +0000 2024,0,@greseella @Greesel_JKT48 Udah kena bedak JOT ...,1.750000e+18,NaN,NaN,in,NaN,0,1,0,https://x.com/nouesophine/status/1748262876828...,1.740000e+18,nouesophine
6214,1.750000e+18,Fri Jan 19 02:11:57 +0000 2024,1,@naim_pasha @officialJKT48 @Y_ChikaJKT48 @F_Fe...,1.750000e+18,NaN,Naelep261,in,"DKI Jakarta, Indonesia",0,0,1,https://x.com/Naelep261/status/174816637217219...,1.290000e+18,Naelep261


PRE PROSESSING DATA

In [115]:
#case folding
df_text = pd.DataFrame({
    'Original_Text': df['full_text'],
    'Lowercased_Text': df['full_text'].str.lower(),
    'tweet_url': df['tweet_url']
})

display(df_text)

,Original_Text,Lowercased_Text,tweet_url
0,Om JOT Liat deh Si up girls ngajak @RFritzy_JK...,om jot liat deh si up girls ngajak @rfritzy_jk...,https://x.com/XJulianto/status/187380500346035...
1,@RFritzy_JKT48 Kalo oshinya JOT masih ga apa a...,@rfritzy_jkt48 kalo oshinya jot masih ga apa a...,https://x.com/Anggithaadr/status/1873788000431...
2,harapan untuk jkt48 yaitu membuat para jot sad...,harapan untuk jkt48 yaitu membuat para jot sad...,https://x.com/blq_abilzz/status/18737590745775...
3,@_rybh @Maffi_f07 @Auwia_JKT48 Itu urusan inte...,@_rybh @maffi_f07 @auwia_jkt48 itu urusan inte...,https://x.com/gagitugie___/status/187375228321...
4,@gagitugie___ @Maffi_f07 @Auwia_JKT48 Menunjuk...,@gagitugie___ @maffi_f07 @auwia_jkt48 menunjuk...,https://x.com/_rybh/status/1873747635750011333
...,...,...,...
6211,Lahkk makasih om jot See u @Greesel_JKT48 http...,lahkk makasih om jot see u @greesel_jkt48 http...,https://x.com/R_Hafidzmuh/status/1748306574794...
6212,@JambuBiji08 @Gracie_JKT48 Harusnya faham JOT om,@jambubiji08 @gracie_jkt48 harusnya faham jot om,https://x.com/zibarrr_/status/1748302786700439566
6213,@greseella @Greesel_JKT48 Udah kena bedak JOT ...,@greseella @greesel_jkt48 udah kena bedak jot ...,https://x.com/nouesophine/status/1748262876828...
6214,@naim_pasha @officialJKT48 @Y_ChikaJKT48 @F_Fe...,@naim_pasha @officialjkt48 @y_chikajkt48 @f_fe...,https://x.com/Naelep261/status/174816637217219...


In [116]:
#cleaning text (Sementara)
import re
def clean_text(text):
    # Menghapus URL
    text = re.sub(r'#', '', text)  # Remove hashtag symbol
    text = re.sub(r'http\S+|www.\S+', '', text ) #remove URl maupun Tautan
    text = re.sub(r'[^\w\s]', '', text) # Rewmove Tanda baca dan karakter khusus
    text = re.sub(r'\d','', text) #remove Angka
    text = re.sub(r'@|w\s','',text) #remove Karakter @ dan pola khusus
    text = re.sub(r'\s+', ' ', text) #remove spasi ganda
    return text


df_text['Cleaned_Text'] = df_text['Lowercased_Text'].apply(clean_text)
display(df_text)

#download csv
# df_text_cleaning.to_csv('df_text_cleaning.csv', index=False)

,Original_Text,Lowercased_Text,tweet_url,Cleaned_Text
0,Om JOT Liat deh Si up girls ngajak @RFritzy_JK...,om jot liat deh si up girls ngajak @rfritzy_jk...,https://x.com/XJulianto/status/187380500346035...,om jot liat deh si up girls ngajak rfritzy_jkt...
1,@RFritzy_JKT48 Kalo oshinya JOT masih ga apa a...,@rfritzy_jkt48 kalo oshinya jot masih ga apa a...,https://x.com/Anggithaadr/status/1873788000431...,rfritzy_jkt kalo oshinya jot masih ga apa apa ...
2,harapan untuk jkt48 yaitu membuat para jot sad...,harapan untuk jkt48 yaitu membuat para jot sad...,https://x.com/blq_abilzz/status/18737590745775...,harapan untuk jkt yaitu membuat para jot sadar...
3,@_rybh @Maffi_f07 @Auwia_JKT48 Itu urusan inte...,@_rybh @maffi_f07 @auwia_jkt48 itu urusan inte...,https://x.com/gagitugie___/status/187375228321...,_rybh maffi_f auwia_jkt itu urusan internal jo...
4,@gagitugie___ @Maffi_f07 @Auwia_JKT48 Menunjuk...,@gagitugie___ @maffi_f07 @auwia_jkt48 menunjuk...,https://x.com/_rybh/status/1873747635750011333,gagitugie___ maffi_f auwia_jkt menunjukan kalo...
...,...,...,...,...
6211,Lahkk makasih om jot See u @Greesel_JKT48 http...,lahkk makasih om jot see u @greesel_jkt48 http...,https://x.com/R_Hafidzmuh/status/1748306574794...,lahkk makasih om jot see u greesel_jkt
6212,@JambuBiji08 @Gracie_JKT48 Harusnya faham JOT om,@jambubiji08 @gracie_jkt48 harusnya faham jot om,https://x.com/zibarrr_/status/1748302786700439566,jambubiji gracie_jkt harusnya faham jot om
6213,@greseella @Greesel_JKT48 Udah kena bedak JOT ...,@greseella @greesel_jkt48 udah kena bedak jot ...,https://x.com/nouesophine/status/1748262876828...,greseella greesel_jkt udah kena bedak jot aja nih
6214,@naim_pasha @officialJKT48 @Y_ChikaJKT48 @F_Fe...,@naim_pasha @officialjkt48 @y_chikajkt48 @f_fe...,https://x.com/Naelep261/status/174816637217219...,naim_pasha officialjkt y_chikajkt f_fenijkt st...


In [117]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [120]:
#Tokenization
df_text['Tokenized_Text'] = df_text['Cleaned_Text'].apply(word_tokenize)
display(df_text)
# display(df_text[['Tokenized_Text','Cleaned_Text']])

,Original_Text,Lowercased_Text,tweet_url,Cleaned_Text,Tokenized_Text
0,Om JOT Liat deh Si up girls ngajak @RFritzy_JK...,om jot liat deh si up girls ngajak @rfritzy_jk...,https://x.com/XJulianto/status/187380500346035...,om jot liat deh si up girls ngajak rfritzy_jkt...,"[om, jot, liat, deh, si, up, girls, ngajak, rf..."
1,@RFritzy_JKT48 Kalo oshinya JOT masih ga apa a...,@rfritzy_jkt48 kalo oshinya jot masih ga apa a...,https://x.com/Anggithaadr/status/1873788000431...,rfritzy_jkt kalo oshinya jot masih ga apa apa ...,"[rfritzy_jkt, kalo, oshinya, jot, masih, ga, a..."
2,harapan untuk jkt48 yaitu membuat para jot sad...,harapan untuk jkt48 yaitu membuat para jot sad...,https://x.com/blq_abilzz/status/18737590745775...,harapan untuk jkt yaitu membuat para jot sadar...,"[harapan, untuk, jkt, yaitu, membuat, para, jo..."
3,@_rybh @Maffi_f07 @Auwia_JKT48 Itu urusan inte...,@_rybh @maffi_f07 @auwia_jkt48 itu urusan inte...,https://x.com/gagitugie___/status/187375228321...,_rybh maffi_f auwia_jkt itu urusan internal jo...,"[_rybh, maffi_f, auwia_jkt, itu, urusan, inter..."
4,@gagitugie___ @Maffi_f07 @Auwia_JKT48 Menunjuk...,@gagitugie___ @maffi_f07 @auwia_jkt48 menunjuk...,https://x.com/_rybh/status/1873747635750011333,gagitugie___ maffi_f auwia_jkt menunjukan kalo...,"[gagitugie___, maffi_f, auwia_jkt, menunjukan,..."
...,...,...,...,...,...
6211,Lahkk makasih om jot See u @Greesel_JKT48 http...,lahkk makasih om jot see u @greesel_jkt48 http...,https://x.com/R_Hafidzmuh/status/1748306574794...,lahkk makasih om jot see u greesel_jkt,"[lahkk, makasih, om, jot, see, u, greesel_jkt]"
6212,@JambuBiji08 @Gracie_JKT48 Harusnya faham JOT om,@jambubiji08 @gracie_jkt48 harusnya faham jot om,https://x.com/zibarrr_/status/1748302786700439566,jambubiji gracie_jkt harusnya faham jot om,"[jambubiji, gracie_jkt, harusnya, faham, jot, om]"
6213,@greseella @Greesel_JKT48 Udah kena bedak JOT ...,@greseella @greesel_jkt48 udah kena bedak jot ...,https://x.com/nouesophine/status/1748262876828...,greseella greesel_jkt udah kena bedak jot aja nih,"[greseella, greesel_jkt, udah, kena, bedak, jo..."
6214,@naim_pasha @officialJKT48 @Y_ChikaJKT48 @F_Fe...,@naim_pasha @officialjkt48 @y_chikajkt48 @f_fe...,https://x.com/Naelep261/status/174816637217219...,naim_pasha officialjkt y_chikajkt f_fenijkt st...,"[naim_pasha, officialjkt, y_chikajkt, f_fenijk..."


In [124]:
# kamus singkatan bahasa indonesia
stopwords = set(stopwords.words('indonesian'))
display(stopwords)

{'ada',
 'adalah',
 'adanya',
 'adapun',
 'agak',
 'agaknya',
 'agar',
 'akan',
 'akankah',
 'akhir',
 'akhiri',
 'akhirnya',
 'aku',
 'akulah',
 'amat',
 'amatlah',
 'anda',
 'andalah',
 'antar',
 'antara',
 'antaranya',
 'apa',
 'apaan',
 'apabila',
 'apakah',
 'apalagi',
 'apatah',
 'artinya',
 'asal',
 'asalkan',
 'atas',
 'atau',
 'ataukah',
 'ataupun',
 'awal',
 'awalnya',
 'bagai',
 'bagaikan',
 'bagaimana',
 'bagaimanakah',
 'bagaimanapun',
 'bagi',
 'bagian',
 'bahkan',
 'bahwa',
 'bahwasanya',
 'baik',
 'bakal',
 'bakalan',
 'balik',
 'banyak',
 'bapak',
 'baru',
 'bawah',
 'beberapa',
 'begini',
 'beginian',
 'beginikah',
 'beginilah',
 'begitu',
 'begitukah',
 'begitulah',
 'begitupun',
 'bekerja',
 'belakang',
 'belakangan',
 'belum',
 'belumlah',
 'benar',
 'benarkah',
 'benarlah',
 'berada',
 'berakhir',
 'berakhirlah',
 'berakhirnya',
 'berapa',
 'berapakah',
 'berapalah',
 'berapapun',
 'berarti',
 'berawal',
 'berbagai',
 'berdatangan',
 'beri',
 'berikan',
 'berikut'

In [ ]:
# masukkan kamus label bahasa indonesia dan bahasa alay besok aja

In [121]:
#Stopwords
stop_words = set(stopwords.words('indonesian'))
df_text['Filtered_Text'] = df_text['Tokenized_Text'].apply(lambda x: [word for word in x if word not in stop_words])

In [123]:
#result hasil
display(df_text[['Tokenized_Text','Cleaned_Text','Filtered_Text']])

,Tokenized_Text,Cleaned_Text,Filtered_Text
0,"[om, jot, liat, deh, si, up, girls, ngajak, rf...",om jot liat deh si up girls ngajak rfritzy_jkt...,"[om, jot, liat, deh, si, up, girls, ngajak, rf..."
1,"[rfritzy_jkt, kalo, oshinya, jot, masih, ga, a...",rfritzy_jkt kalo oshinya jot masih ga apa apa ...,"[rfritzy_jkt, kalo, oshinya, jot, ga, nder, hehe]"
2,"[harapan, untuk, jkt, yaitu, membuat, para, jo...",harapan untuk jkt yaitu membuat para jot sadar...,"[harapan, jkt, jot, sadar, fans, apaapa, gaaka..."
3,"[_rybh, maffi_f, auwia_jkt, itu, urusan, inter...",_rybh maffi_f auwia_jkt itu urusan internal jo...,"[_rybh, maffi_f, auwia_jkt, urusan, internal, ..."
4,"[gagitugie___, maffi_f, auwia_jkt, menunjukan,...",gagitugie___ maffi_f auwia_jkt menunjukan kalo...,"[gagitugie___, maffi_f, auwia_jkt, menunjukan,..."
...,...,...,...
6211,"[lahkk, makasih, om, jot, see, u, greesel_jkt]",lahkk makasih om jot see u greesel_jkt,"[lahkk, makasih, om, jot, see, u, greesel_jkt]"
6212,"[jambubiji, gracie_jkt, harusnya, faham, jot, om]",jambubiji gracie_jkt harusnya faham jot om,"[jambubiji, gracie_jkt, faham, jot, om]"
6213,"[greseella, greesel_jkt, udah, kena, bedak, jo...",greseella greesel_jkt udah kena bedak jot aja nih,"[greseella, greesel_jkt, udah, kena, bedak, jo..."
6214,"[naim_pasha, officialjkt, y_chikajkt, f_fenijk...",naim_pasha officialjkt y_chikajkt f_fenijkt st...,"[naim_pasha, officialjkt, y_chikajkt, f_fenijk..."


In [ ]:
#Stemming
factory = StemmerFactory()
stemmer = factory.create_stemmer()
df_text['Stemmed_Text'] = df_text['Filtered_Text'].apply(lambda x: [stemmer.stem(word) for word in x])]
display(df_text)

PELABELAN DATA

In [ ]:
# Pelabelan Data

In [ ]:
# extrasi fitur

In [ ]:
# klasifikasi sentimen

In [ ]:
# evaluasi hasil


In [ ]:
# visualisasi dan hasilnya